### 5-1

In [17]:
# 필요한 라이브러리 임포트
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import chain
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
import re

# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])


sk


In [18]:

# 1. 벡터 DB 로드
def load_vector_db(db_path="./db/cafe_db"):
    """
    저장된 FAISS 벡터 데이터베이스를 로드합니다.
    """
    print(f"'{db_path}' 경로에서 FAISS 벡터 DB 로드 중...")
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    print("FAISS 벡터 DB 로드 완료.")
    return vectorstore

# 벡터 DB 인스턴스 로드 (전역적으로 사용)
cafe_vector_db = load_vector_db()



'./db/cafe_db' 경로에서 FAISS 벡터 DB 로드 중...
FAISS 벡터 DB 로드 완료.


In [20]:

# 2. 도구 정의

# a) tavily_search_func
tavily_search_tool = TavilySearchResults()

# b) wiki_summary
wikipedia_wrapper = WikipediaAPIWrapper()
@tool
def wiki_summary(query: str) -> str:
    """
    위키피디아에서 일반 지식 검색 및 요약.
    입력: 검색 주제 (str)
    출력: 요약된 정보 (str)
    """
    print(f"Calling Wikipedia tool with query: {query}")
    try:
        summary = wikipedia_wrapper.run(query)
        return summary
    except Exception as e:
        return f"위키피디아 검색 중 오류 발생: {e}"

# c) db_search_cafe_func
@tool
def db_search_cafe_func(query: str) -> str:
    """
    로컬 카페 메뉴 DB에서 메뉴 관련 정보 검색.
    입력: 메뉴 관련 쿼리 (str)
    출력: 관련 메뉴 정보 (str)
    사용 예: 아메리카노의 가격, 재료, 특징 등
    """
    print(f"Calling DB search tool with query: {query}")
    docs = cafe_vector_db.similarity_search(query, k=1) # 가장 유사한 문서 1개 검색
    if docs:
        # Document 객체의 page_content를 반환
        return docs[0].page_content
    return "해당 메뉴에 대한 정보를 찾을 수 없습니다."

# 모든 도구 리스트
tools = [tavily_search_tool, wiki_summary, db_search_cafe_func]

# 3. LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 4. 프롬프트 정의
system_prompt_content = (
    "당신은 카페 메뉴에 대한 질문에 답변하고, 필요시 웹 검색이나 위키피디아를 활용하는 친절한 AI 어시스턴트입니다."
    "메뉴 정보는 'db_search_cafe_func' 도구를 사용하고, 일반적인 지식은 'wiki_summary'를, 최신 정보나 웹 검색은 'tavily_search_func'를 사용하세요."
)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_content),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 5. LLM에 도구 바인딩
agent_llm = llm.bind_tools(tools)

# 6. 간단한 도구 호출 체인 구현
@chain
def tool_calling_chain(input_data: dict):
    """
    사용자 질문을 받아 LLM이 도구를 선택하고 실행하며, 결과를 종합하여 최종 답변을 생성하는 체인.
    """
    query = input_data["input"]
    chat_history = input_data.get("chat_history", [])
    
    agent_scratchpad = [] 

    prompt_values = {
        "input": query,
        "chat_history": chat_history,
        "agent_scratchpad": agent_scratchpad
    }
    
    full_prompt_message = prompt.invoke(prompt_values)
    
    ai_message = agent_llm.invoke(full_prompt_message)
    
    # Step 2: 도구 호출 결과 확인 및 실행
    # 여기가 수정된 부분입니다: tool_call_dict로 접근하도록 변경
    if ai_message.tool_calls:
        print(f"\n[Tool Call Detected]: {ai_message.tool_calls}")
        tool_results = []
        for tool_call_dict in ai_message.tool_calls: # tool_call_dict로 이름 변경
            # dict 객체에서 'name', 'args', 'id' 키로 접근
            tool_name = tool_call_dict.get('name') 
            tool_args = tool_call_dict.get('args')
            tool_id = tool_call_dict.get('id')

            if not tool_name: # 'name' 키가 없는 경우를 대비 (예외 처리)
                print(f"Warning: Tool call dictionary missing 'name' key: {tool_call_dict}. Skipping this tool call.")
                continue

            result = None
            try:
                if tool_name == tavily_search_tool.name:
                    # tool_args가 None일 수 있으므로 .get('query') 호출 시 기본값 지정
                    result = tavily_search_tool.invoke(tool_args.get("query") if tool_args else "")
                elif tool_name == wiki_summary.name:
                    result = wiki_summary.invoke(tool_args.get("query") if tool_args else "")
                elif tool_name == db_search_cafe_func.name:
                    result = db_search_cafe_func.invoke(tool_args.get("query") if tool_args else "")
                else:
                    result = f"알 수 없는 도구: {tool_name}"
            except Exception as e:
                result = f"도구 {tool_name} 실행 중 오류 발생: {e}"
            
            # ToolMessage 생성 시 tool_id를 전달
            tool_results.append(ToolMessage(content=str(result), tool_call_id=tool_id))
        
        # Step 3: 도구 실행 결과를 포함하여 LLM 재호출하여 최종 답변 생성
        updated_agent_scratchpad = [ai_message] + tool_results

        re_invoke_prompt_values = {
            "input": query,
            "chat_history": chat_history,
            "agent_scratchpad": updated_agent_scratchpad
        }
        
        re_invoke_full_prompt_message = prompt.invoke(re_invoke_prompt_values)
        
        final_response_message = agent_llm.invoke(re_invoke_full_prompt_message)
        return final_response_message.content
    else:
        # 도구 호출이 없는 경우 LLM의 원래 답변 반환
        return ai_message.content

# 7. 세션 기록 관리 및 RunnableWithMessageHistory 바인딩
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_chain = RunnableWithMessageHistory(
    tool_calling_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 8. 테스트 질문 처리
if __name__ == "__main__":
    print("\n--- 카페 AI 어시스턴트 시작 (종료하려면 'exit' 또는 'quit' 입력) ---")

    # 세션 ID를 하나로 유지하여 이전 대화 기록을 활용하도록 설정 (선택 사항)
    session_id = "user_session_123" 

    while True:
        user_input = input("\n당신의 질문을 입력하세요: ")
        
        if user_input.lower() in ["exit", "quit", "종료"]:
            print("AI 어시스턴트를 종료합니다.")
            break
        
        print(f"\n[사용자 질문]: {user_input}")
        
        try:
            response = conversational_chain.invoke(
                {"input": user_input},
                config={"configurable": {"session_id": session_id}}
            )
            print(f"[AI 답변]: {response}")
        except Exception as e:
            print(f"[오류 발생]: {e}")
            print("오류가 발생하여 질문 처리를 중단합니다. 관리자에게 문의하세요.")
            # 오류 발생 시 루프를 종료하거나 계속 진행할지 선택할 수 있습니다.
            # 여기서는 오류가 나더라도 계속 질문을 받을 수 있도록 break를 제거합니다.

    print("\n--- AI 어시스턴트 종료 ---")


--- 카페 AI 어시스턴트 시작 (종료하려면 'exit' 또는 'quit' 입력) ---

[사용자 질문]: 아메리카노의 가격은 얼마야?

[Tool Call Detected]: [{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노 가격'}, 'id': 'call_3lNCQyZVz9niH7jAJYQJZHJx', 'type': 'tool_call'}]
Calling DB search tool with query: 아메리카노 가격
[AI 답변]: 아메리카노의 가격은 ₩4,500입니다. 이 음료는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, 원두 본연의 맛을 잘 느낄 수 있으며 깔끔하고 깊은 풍미가 특징입니다. 필요에 따라 설탕이나 시럽을 추가할 수 있습니다.

[사용자 질문]: 고마워 끝!
[AI 답변]: 천만에요! 궁금한 점이 있으면 언제든지 말씀해 주세요. 좋은 하루 되세요! 😊
AI 어시스턴트를 종료합니다.

--- AI 어시스턴트 종료 ---


### 5-2

In [21]:
# 필요한 라이브러리 임포트
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import chain
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
import re

# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])


sk


In [22]:


# 1. 벡터 DB 로드
def load_vector_db(db_path="./db/cafe_db"):
    """
    저장된 FAISS 벡터 데이터베이스를 로드합니다.
    """
    print(f"'{db_path}' 경로에서 FAISS 벡터 DB 로드 중...")
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    print("FAISS 벡터 DB 로드 완료.")
    return vectorstore

# 벡터 DB 인스턴스 로드 (전역적으로 사용)
cafe_vector_db = load_vector_db()


'./db/cafe_db' 경로에서 FAISS 벡터 DB 로드 중...
FAISS 벡터 DB 로드 완료.


In [23]:
# 2. 도구 정의

# a) tavily_search_func
tavily_search_tool = TavilySearchResults()

# b) wiki_summary
wikipedia_wrapper = WikipediaAPIWrapper()
@tool
def wiki_summary(query: str) -> str:
    """
    위키피디아에서 일반 지식 검색 및 요약.
    입력: 검색 주제 (str)
    출력: 요약된 정보 (str)
    """
    print(f"Calling Wikipedia tool with query: {query}")
    try:
        summary = wikipedia_wrapper.run(query)
        return summary
    except Exception as e:
        return f"위키피디아 검색 중 오류 발생: {e}"

# c) db_search_cafe_func
@tool
def db_search_cafe_func(query: str) -> str:
    """
    로컬 카페 메뉴 DB에서 메뉴 관련 정보 검색.
    입력: 메뉴 관련 쿼리 (str)
    출력: 관련 메뉴 정보 (str)
    사용 예: 아메리카노의 가격, 재료, 특징 등
    """
    print(f"Calling DB search tool with query: {query}")
    docs = cafe_vector_db.similarity_search(query, k=1) # 가장 유사한 문서 1개 검색
    if docs:
        # Document 객체의 page_content를 반환
        return docs[0].page_content
    return "해당 메뉴에 대한 정보를 찾을 수 없습니다."

# 모든 도구 리스트
tools = [tavily_search_tool, wiki_summary, db_search_cafe_func]

# 3. LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 4. 프롬프트 정의
system_prompt_content = (
    "당신은 카페 메뉴에 대한 질문에 답변하고, 필요시 웹 검색이나 위키피디아를 활용하는 친절한 AI 어시스턴트입니다."
    "메뉴 정보는 'db_search_cafe_func' 도구를 사용하고, 일반적인 지식은 'wiki_summary'를, 최신 정보나 웹 검색은 'tavily_search_func'를 사용하세요."
    "도구를 사용해야 할 시점을 정확하게 판단하고, 불필요한 도구 호출은 삼가세요. 도구를 사용하지 않고 답변할 수 있는 질문에는 바로 답변하세요."
)

# Few-shot 예제 정의 (메시지 객체 리스트)
# AI의 ToolCall 형식은 LLM이 실제로 출력하는 딕셔너리 형태를 가정하여 작성했습니다.
few_shot_examples = [
    HumanMessage(content="아메리카노 가격 얼마인가요?"),
    AIMessage(
        content="",
        tool_calls=[
            {"name": "db_search_cafe_func", "args": {"query": "아메리카노 가격"}, "id": "call_1"}
        ]
    ),
    ToolMessage(
        content="아메리카노: ₩4,500. 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다.",
        tool_call_id="call_1"
    ),
    AIMessage(content="아메리카노는 ₩4,500이며, 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다."),

    HumanMessage(content="최신 커피 트렌드가 궁금해요."),
    AIMessage(
        content="",
        tool_calls=[
            {"name": "tavily_search_tool", "args": {"query": "2025년 최신 커피 트렌드"}, "id": "call_2"}
        ]
    ),
    ToolMessage(
        content="검색 결과: 2025년에는 지속 가능한 커피, 콜드브루의 인기 지속, RTD(Ready To Drink) 커피 시장 성장 등의 트렌드가 예상됩니다.",
        tool_call_id="call_2"
    ),
    AIMessage(content="2025년 최신 커피 트렌드로는 지속 가능한 커피, 콜드브루의 인기 지속, RTD(Ready To Drink) 커피 시장 성장 등이 예상됩니다."),
    
    HumanMessage(content="커피는 언제부터 마시기 시작했나요?"),
    AIMessage(
        content="",
        tool_calls=[
            {"name": "wiki_summary", "args": {"query": "커피의 역사"}, "id": "call_3"}
        ]
    ),
    ToolMessage(
        content="위키피디아 요약: 커피는 9세기 에티오피아에서 처음 발견된 것으로 추정되며, 이후 이슬람 세계를 거쳐 유럽으로 전파되었습니다.",
        tool_call_id="call_3"
    ),
    AIMessage(content="커피는 9세기 에티오피아에서 처음 발견된 것으로 추정되며, 이후 이슬람 세계를 거쳐 유럽으로 전파되었습니다."),

    HumanMessage(content="안녕하세요?"), # 도구 호출이 필요 없는 일반 대화 예시
    AIMessage(content="안녕하세요! 궁금한 카페 메뉴나 커피에 대해 무엇이든 물어보세요."),

    HumanMessage(content="오늘 날씨는 어떤가요?"), # 도구 호출이 필요 없는 일반 대화 예시 (아는 정보가 아님)
    AIMessage(content="저는 카페 메뉴와 관련된 질문에 답변해 드릴 수 있습니다. 날씨 정보는 제공해 드릴 수 없어요."),
]

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_content),
        # 여기에 Few-shot 예제 메시지들을 추가합니다.
        *few_shot_examples,
        # 이후에는 실제 대화 기록과 현재 사용자 입력, 에이전트 스크래치패드가 이어집니다.
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 5. LLM에 도구 바인딩
agent_llm = llm.bind_tools(tools)

# 6. 간단한 도구 호출 체인 구현
@chain
def tool_calling_chain(input_data: dict):
    """
    사용자 질문을 받아 LLM이 도구를 선택하고 실행하며, 결과를 종합하여 최종 답변을 생성하는 체인.
    """
    query = input_data["input"]
    chat_history = input_data.get("chat_history", [])
    
    agent_scratchpad = [] 

    prompt_values = {
        "input": query,
        "chat_history": chat_history,
        "agent_scratchpad": agent_scratchpad
    }
    
    full_prompt_message = prompt.invoke(prompt_values)
    
    ai_message = agent_llm.invoke(full_prompt_message)
    
    # Step 2: 도구 호출 결과 확인 및 실행
    if ai_message.tool_calls:
        print(f"\n[Tool Call Detected]: {ai_message.tool_calls}")
        tool_results = []
        for tool_call_dict in ai_message.tool_calls:
            tool_name = tool_call_dict.get('name') 
            tool_args = tool_call_dict.get('args')
            tool_id = tool_call_dict.get('id')

            if not tool_name:
                print(f"Warning: Tool call dictionary missing 'name' key: {tool_call_dict}. Skipping this tool call.")
                continue

            result = None
            try:
                if tool_name == tavily_search_tool.name:
                    result = tavily_search_tool.invoke(tool_args.get("query") if tool_args else "")
                elif tool_name == wiki_summary.name:
                    result = wiki_summary.invoke(tool_args.get("query") if tool_args else "")
                elif tool_name == db_search_cafe_func.name:
                    result = db_search_cafe_func.invoke(tool_args.get("query") if tool_args else "")
                else:
                    result = f"알 수 없는 도구: {tool_name}"
            except Exception as e:
                result = f"도구 {tool_name} 실행 중 오류 발생: {e}"
            
            tool_results.append(ToolMessage(content=str(result), tool_call_id=tool_id))
        
        # Step 3: 도구 실행 결과를 포함하여 LLM 재호출하여 최종 답변 생성
        updated_agent_scratchpad = [ai_message] + tool_results

        re_invoke_prompt_values = {
            "input": query,
            "chat_history": chat_history,
            "agent_scratchpad": updated_agent_scratchpad
        }
        
        re_invoke_full_prompt_message = prompt.invoke(re_invoke_prompt_values)
        
        final_response_message = agent_llm.invoke(re_invoke_full_prompt_message)
        return final_response_message.content
    else:
        # 도구 호출이 없는 경우 LLM의 원래 답변 반환
        return ai_message.content

# 7. 세션 기록 관리 및 RunnableWithMessageHistory 바인딩
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_chain = RunnableWithMessageHistory(
    tool_calling_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 8. 테스트 질문 처리 (사용자 입력으로 변경)
if __name__ == "__main__":
    print("\n--- 카페 AI 어시스턴트 시작 (종료하려면 'exit' 또는 'quit' 입력) ---")

    session_id = "user_session_fewshot_1" # Few-shot 테스트를 위한 새 세션 ID

    while True:
        user_input = input("\n[당신의 질문]: ")
        
        if user_input.lower() in ["exit", "quit", "종료"]:
            print("AI 어시스턴트를 종료합니다.")
            break
        
        try:
            response = conversational_chain.invoke(
                {"input": user_input},
                config={"configurable": {"session_id": session_id}}
            )
            print(f"[AI 답변]: {response}")
        except Exception as e:
            print(f"[오류 발생]: {e}")
            print("오류가 발생하여 질문 처리를 중단할 수 있습니다. 스크립트를 다시 시작하거나 관리자에게 문의하세요.")

    print("\n--- AI 어시스턴트 종료 ---")


--- 카페 AI 어시스턴트 시작 (종료하려면 'exit' 또는 'quit' 입력) ---

[Tool Call Detected]: [{'name': 'wiki_summary', 'args': {'query': '라떼의 역사'}, 'id': 'call_0zrBaXB7tK3aMaOYhFiHXnok', 'type': 'tool_call'}]
Calling Wikipedia tool with query: 라떼의 역사
[AI 답변]: 라떼의 역사에 대한 구체적인 정보를 찾지 못했습니다. 하지만 일반적으로 라떼는 에스프레소와 스팀 밀크를 결합한 음료로, 20세기 초 이탈리아에서 유래되었습니다. 라떼는 커피 문화가 발전하면서 전 세계적으로 인기를 얻었으며, 다양한 변형이 생겨났습니다. 더 자세한 정보가 필요하시다면 다른 질문을 해주세요!

[Tool Call Detected]: [{'name': 'db_search_cafe_func', 'args': {'query': '카페라떼 특성'}, 'id': 'call_8xqIeYGyenfKQq1yWDbHkz29', 'type': 'tool_call'}]
Calling DB search tool with query: 카페라떼 특성
[AI 답변]: 카페라떼의 특성은 다음과 같습니다:

- **가격**: ₩5,500
- **주요 원료**: 에스프레소, 스팀 밀크
- **설명**: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 또한, 라떼 아트로 시각적 즐거움도 제공합니다.
[AI 답변]: 천만에요! 언제든지 궁금한 점이 있으면 다시 찾아주세요. 좋은 하루 되세요! 😊
AI 어시스턴트를 종료합니다.

--- AI 어시스턴트 종료 ---
